Source: https://github.com/UKPLab/sentence-transformers

In [7]:
!pip install pycarol

In [ ]:
!pip install -U sentence-transformers

In [121]:
from pycarol import Carol, Staging, ApiKeyAuth, Storage, PwdAuth
import pandas as pd
import pickle
import numpy as np
import re
import math

In [122]:
login_dont = Carol(domain='monitoriaqa',
app_name=' ',
organization='totvs',
auth=PwdAuth('fmendes@totvs.com.br', 'Intxdx70248821*'))

In [136]:
df = Staging(login_dont).fetch_parquet(staging_name='articles',
                        connector_name='carol_connect_zerado',
                        cds=True
                        )

100%|██████████| 24/24 [00:30<00:00,  1.27s/it]


In [137]:
df = df.astype(str)

In [138]:
df.shape

(37290, 26)

In [139]:
df = df[df['title'].str.contains('CST - LGPD')]

In [140]:
df.shape

(53, 26)

In [141]:
df.title.unique()

array(['CST - LGPD - Atualização de dados de Parceiros',
       'CST - LGPD - Acesso aos dados de fornecedores inativos',
       'CST - LGPD - Solicitar exclusão, anonimização e/ou bloqueio de dados de Fornecedor Inativo',
       'CST - LGPD - Tempo de armazenamento de dados posterior ao cancelamento de contrato',
       'CST - LGPD - Anonimização, bloqueio ou exclusão dados Parceiro',
       'CST - LGPD - Acesso aos meus dados pessoais  cadastrados na TOTVS',
       'CST - LGPD - Armazenamento de dados após desligamento',
       'CST - LGPD - Dados de candidatura das vagas na TOTVS',
       'CST - LGPD - Bloqueio de dados com processo judicial em andamento',
       'CST - LGPD - Anonimização, bloqueio ou exclusão dos dados no cliente',
       'CST - LGPD - Compartilhamento de dados pessoais',
       'CST - LGPD - Atualização de dados pessoais do cliente',
       'CST - LGPD - Acesso aos dados pessoais cadastrados na TOTVS',
       'CST - LGPD - Solicitar anonimização por funcionário a

In [13]:
df = df.replace('', np.nan)

In [14]:
df = df[(df.source_locale == 'pt-br') & (~df.body.isnull())]

In [15]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    return " ".join(re.sub(r'\s([?.!"](?:\s|$))', r'\1', text).strip().split())

In [16]:
def get_question(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>D(ú|u)vida).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
        
    m = re.search('(?<=<strong>Ocorr(ê|e)ncia).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
    
    return np.nan

In [17]:
def get_question_type(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>D(ú|u)vida).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return 'question'

    m = re.search('(?<=<strong>Ocorr(ê|e)ncia).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return 'occurrence'
    
    return np.nan

In [18]:
def get_environment(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?=<strong>Ambiente).*?(?=<strong>Solu(ç|c)(ã|a)o)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return remove_html_tags(m.group(0))

In [19]:
def get_solution(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>Solu(ç|c)(ã|a)o)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return m.group(0)

In [20]:
def get_sanitized_solution(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>Solu(ç|c)(ã|a)o)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return remove_html_tags(m.group(0))

In [21]:
df['question'] = df.body.apply(get_question)
df['question_type'] = df.body.apply(get_question_type)
df['environment'] = df.body.apply(get_environment)
df['solution'] = df.body.apply(get_solution)
df['sanitized_solution'] = df.body.apply(get_sanitized_solution)

/Users/felipe/opt/miniconda3/envs/bert/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Flags not at the start of the expression '(?<=<strong>Solu(ç|c' (truncated)
  """


In [22]:
df = df.replace('', np.nan)

In [23]:
df.head()

,__v,author_id,body,brand_id,brand_name,comments_disabled,created_at,draft,edited_at,html_url,...,updated,updated_at,url,vote_count,vote_sum,question,question_type,environment,solution,sanitized_solution
3603,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-05-21T17:42:36.000Z,True,2020-05-28T15:11:04.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-05-28T16:53:39.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Como faço para atualizar os meus dados pessoais?,question,AmbienteCentro de serviço - Lei Geral de Prote...,</strong><strong></strong>Caso tenha algum dad...,Caso tenha algum dado que esteja desatualizado...
3606,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-05-21T17:56:00.000Z,True,2020-05-28T14:55:52.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-05-28T16:54:16.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,O que é a política da LGPD?,question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong>A TOTVS tem como pri...,"A TOTVS tem como prioridade proteger, preserva..."
3608,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-05-21T17:49:08.000Z,True,2020-05-28T15:04:58.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-05-28T16:55:03.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Quais são os papéis dentro da LGPD?,question,AmbienteCentro de serviço - Lei Geral de Prote...,</strong></p><ul><li><strong>Titular dos Dados...,Titular dos Dados é a pessoa física a quem se ...
3618,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,False,2020-05-21T17:54:09.000Z,True,2020-05-28T14:58:22.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-05-28T16:54:27.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Qual o objetivo do canal de atendimento para d...,question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong>O canal de atendimen...,O canal de atendimento para direitos dos titul...
3623,0,6689975327.0,<p><strong>Dúvida</strong><strong><br></strong...,1509248,TOTVS,True,2020-05-21T17:38:50.000Z,True,2020-05-28T16:25:17.000Z,https://centraldeatendimento.totvs.com/hc/pt-b...,...,True,2020-05-28T16:53:15.000Z,https://totvsexterno.zendesk.com/api/v2/help_c...,0,0,Qual o papel do consentimento no direito dos t...,question,AmbienteCentro de Serviço - Lei Geral de Prote...,</strong><strong></strong>O consentimento perm...,O consentimento permite que o titular tenha ac...


In [24]:
df = df.dropna(subset=['question', 'question_type', 'environment', 'solution'])

In [25]:
df = df.reset_index(drop=True)

In [26]:
df['question'].count()

14

In [27]:
df['id'] = df['id'].apply(lambda x: x.replace('.0', ''))

In [28]:
df.iloc[0].id

'360048955753'

In [45]:
dupl = df[df.duplicated(['question'])]

In [46]:
dupl[['question', 'brand_id', 'label_names', 'title', 'html_url']].to_excel('articles_with_duplicate_questions.xlsx', index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
dupl.head()[['question', 'brand_id', 'label_names', 'title']]

In [ ]:
dupl.iloc[1].html_url

In [90]:
hifen = df[df.question.str.contains('((?:\w+-)+\w+)', regex=True)]

<ipython-input-90-0571c2c7d7f7>:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  hifen = df[df.question.str.contains('((?:\w+-)+\w+)', regex=True)]


In [91]:
hifen.shape

(3173, 31)

In [92]:
hifen.sample(5)[['question']]

,question
9567,Rejeição 1017 - NF-e Emitido em ambiente de pr...
6189,S-1200 - Como informar 13º Salário para Pró-la...
12263,MDTA640 - THREAD ERROR invalid class MDT640EVE...
4174,Quais os fontes envolvidos para cancelamento d...
4375,Como consultar no site do eSocial dados do eve...


In [93]:
hifen.iloc[3].question

'É possível alterar o endereço de e-mail de um usuário no Identity?'

In [94]:
hifen[['question', 'brand_id', 'label_names', 'title', 'html_url']].to_excel('articles_with_hyphenated_words_in_questions.xlsx', index=False)

In [29]:
target = Carol(domain='mendes',
app_name='searchlgpd',
organization='datascience',
auth=ApiKeyAuth('bf56760357424b60ae9558bcf9f090c2'),
connector_id='175e781f1cce470eb0457323430f1407')

In [37]:
target_stag = Staging(target)

In [97]:
target_stag.send_data(staging_name='articles', connector_id='175e781f1cce470eb0457323430f1407', data=df,
                             crosswalk_auto_create=['idinternalmdb'], auto_create_schema=True)

<ipython-input-97-33f5a54b4626>:1: DeprecationWarning: fields_dict will be deprecated, use `data`
  target_stag.send_data(staging_name='articles', connector_id='175e781f1cce470eb0457323430f1407', data=df,


provided crosswalk  ['idinternalmdb']


### Get sentence embeddings

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [30]:
# Embed a list of sentences
sentence_embeddings = model.encode(df.question.values)

In [31]:
sentence_embeddings[:10]

[array([-0.05934872,  0.02180461, -0.00917931,  0.02191107, -0.06955869,
         0.03660789, -0.02371017, -0.01244595,  0.01701423, -0.10362726,
         0.0024973 ,  0.00838947, -0.03465331, -0.00456053, -0.02349719,
         0.05757834,  0.02949427,  0.03951875, -0.07875644,  0.03221134,
        -0.06575292, -0.05158757,  0.03920632, -0.02295877, -0.00462366,
        -0.01261519, -0.01666118,  0.0088603 ,  0.00061523,  0.02016558,
         0.0383406 ,  0.03902902, -0.07348219, -0.01400102,  0.05533921,
        -0.0121338 ,  0.01385968, -0.03790585,  0.01737368,  0.03085956,
        -0.00823965, -0.00037513,  0.00869321,  0.02421395,  0.02302777,
        -0.03252267, -0.04679887,  0.00788235,  0.0174855 , -0.04680035,
        -0.01018852,  0.04563259, -0.1372602 , -0.00300492,  0.04492883,
        -0.00197007,  0.05702331, -0.00595822,  0.08240177,  0.01823072,
        -0.02495244, -0.01176428,  0.00283476,  0.04163041,  0.03612917,
         0.03737028, -0.05185568, -0.0416397 ,  0.0

In [32]:
len(sentence_embeddings)

14

In [33]:
id_list = df['id'].values

In [34]:
len(id_list)

14

In [35]:
index_to_question_id_mapping = dict(enumerate(id_list))
question_id_to_index_mapping = {j: i for i,j in index_to_question_id_mapping.items()} 

### Save objects in Storage

In [36]:
stg = Storage(target)

In [37]:
def save_object_to_storage(obj, filename):
    with open(filename, "bw") as f:
        pickle.dump(obj, f)
    stg.save(filename, obj, format='pickle')

In [38]:
save_object_to_storage(sentence_embeddings, 'sentence_embeddings')

In [39]:
save_object_to_storage(index_to_question_id_mapping, 'index_to_question_id_mapping')

In [40]:
save_object_to_storage(question_id_to_index_mapping, 'question_id_to_index_mapping')

In [1]:
!python --version

Python 3.8.2


In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler

In [2]:
# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [21]:
import pickle

In [5]:
with open('encoder', "bw") as f:
    pickle.dump(model, f)

/Users/felipe/opt/miniconda3/envs/recommendation/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [22]:
sentence_embeddings = pickle.load(open("sentence_embeddings", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'sentence_embeddings'

In [23]:
sentence_embeddings[:10]

NameError: name 'sentence_embeddings' is not defined

In [ ]:
df.index

In [28]:
df.iloc[12].question

"Após configurar HTTPS no fluig, ao acessá-lo pelo aplicativo fluig Mobile e informar servidor de conexão é apresentado o erro 'Falha na comunicação'."

In [144]:
def get_most_similar_question(query):

    query_vec = model.encode([query])
    k = 5

    # compute normalized dot product as score
    score = np.sum(query_vec[0] * sentence_embeddings, axis=1) / np.linalg.norm(sentence_embeddings, axis=1)
    topk_scores = np.sort(score)[::-1]
    topk_idx = np.argsort(score)[::-1]
    return topk_idx[:k], topk_scores[:k]

In [119]:
#size = np.int32(np.floor(0.2 * questions.shape[0]))
size = 100

indices = np.random.choice(
    df.question.index, 
    replace=False,
    size=size
).tolist()

In [71]:
questions = []

In [93]:
df.iloc[indices[9]].question

'Solicito uma função para conversão de um arquivo .doc para .PDF'

In [94]:
questions.append('função para converter arquivo doc para pdf')

In [97]:
len(questions)

10

In [145]:
get_most_similar_question('adicionar transportadora nas notas fiscais geradas na rotina 1322')

(array([ 4270, 23653, 14174, 26242, 27618]),
 array([0.64584833, 0.60263   , 0.5729962 , 0.54728776, 0.54643303],
       dtype=float32))

In [150]:
df.iloc[4270].question

'Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.'

In [126]:
for i, query in enumerate(questions):
    raw_query = df.iloc[indices[i]].question
    most_similar = get_most_similar_question(query)
    if most_similar != i:
        print(f'{query}')
        print(f'{indices[i]} - {raw_query}')
        print(f'{most_similar} - {df.iloc[most_similar].question}')

0.645848274230957 - 0.645848274230957
0.6026300191879272 - 0.6026300191879272
0.5729961395263672 - 0.5729961395263672
0.5472877025604248 - 0.5472877025604248
0.5464330315589905 - 0.5464330315589905
0.5448733568191528 - 0.5448733568191528
0.5358591079711914 - 0.5358591079711914
0.531542956829071 - 0.531542956829071
0.5271217823028564 - 0.5271217823028564
0.5169530510902405 - 0.5169530510902405
adicionar transportadora nas notas fiscais geradas na rotina 1322
17835 - Por que um determinado lançamento não é apresentado na rotina 723?
4270 - Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.
0.5572085976600647 - 0.5572085976600647
0.5351568460464478 - 0.5351568460464478
0.5280529856681824 - 0.5280529856681824
0.5269234776496887 - 0.5269234776496887
0.5140586495399475 - 0.5140586495399475
0.5049834847450256 - 0.5049834847450256
0.49780139327049255 - 0.49780139327049255
0.4954758882522583 - 0.4954758882522583
0.49390849471092224 - 0.4939

In [115]:
df.iloc[23587].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360035902474-LG-PAT-Formas-de-transfer%C3%AAncia-de-um-bem-para-outro-na-mesma-empresa'

In [114]:
df.iloc[17132].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360025943751-LG-PAT-Como-efetuar-a-transfer%C3%AAncia-de-bens-entre-empresas'

In [112]:
df.iloc[1527].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360045420414-WINT-Como-utilizar-a-rotina-2039-Cadastro-de-Finalizadora'

In [32]:
from pycarol import Query

In [33]:
from pycarol.filter import Filter, TYPE_FILTER, TERMS_FILTER

In [30]:
def get_similar_questions(model, sentence_embeddings, query, k):
    query_vec = model.encode([query])
    score = np.sum(query_vec[0] * sentence_embeddings, axis=1) / np.linalg.norm(sentence_embeddings, axis=1)
    print(score[:100])
    topk_scores = np.sort(score)[::-1]
    topk_idx = np.argsort(score)[::-1]
    print(topk_idx[:k], topk_scores[:k])
    return list(topk_idx[:k]), list(topk_scores[:k])

In [31]:
def get_questions_by_ids(question_ids):
    dm_name = 'articles'
    query_filter = Filter.Builder().must(TYPE_FILTER(value=f'{dm_name}Golden')) \
        .must(TERMS_FILTER(key='mdmGoldenFieldAndValues.id.raw', value=question_ids)).build().to_json()
    page_size=1000
    return Query(target, page_size=page_size).query(query_filter).go().results

In [34]:
def query(query):
    k = 5
    topk_idx, topk_scores = get_similar_questions(model, sentence_embeddings, query, k)
    question_ids = [index_to_question_id_mapping[idx] for idx in topk_idx]
    results = get_questions_by_ids(question_ids)
    response = ''
    for i, question_id in enumerate(question_ids):
        result = [result for result in results if result['id'] == question_id][0]
        response = response + 'Pergunta similar: ' + result['question'] + '\n'
        response = response + 'Resposta: ' + result['solution'] + '\n'
        response = response + 'Score: ' + str(topk_scores[topk_idx.index(question_id_to_index_mapping[question_id])])
        if i != 4:
            response = response +  '\n\n'
    # for i, idx in enumerate(topk_idx):
    #     question_id = index_to_question_id_mapping[idx]
    #     question = get_question_by_id(question_id)[0]
    #     response = response + 'Pergunta similar: ' + question.question + '\n'
    #     response = response + 'Resposta: ' + question.solution + '\n'
    #     response = response + 'Score: ' + topk_scores[i]
    #     if i != 4:
    #         response = response +  '\n\n'
    return {'response': response}

In [35]:
query('adicionar transportadora nas notas fiscais geradas na rotina 1322')

[ 0.02887835 -0.00526772 -0.02707283 -0.03857228  0.03806132  0.04284458
  0.0201705   0.06050855  0.05173742  0.03805823  0.02416999  0.04116369
 -0.06460869  0.09727325  0.08634991  0.01915933  0.08498693 -0.05493666
  0.02492339 -0.00886754  0.10098135  0.06362017  0.0776341  -0.02791644
  0.01163275  0.0192889  -0.01115312 -0.02294862 -0.00829241 -0.03112706
  0.0017001  -0.04119982 -0.01010577 -0.00929159  0.05805634  0.00259353
 -0.02442231 -0.040168    0.00310044 -0.05039007 -0.00759398 -0.03024388
 -0.05424359  0.11716861  0.05606491  0.02613153  0.07075207 -0.02040705
  0.00579199  0.05455872 -0.05031964  0.01694646  0.04972199  0.01136606
  0.01005764 -0.02961715  0.05143109  0.02394645 -0.05122325 -0.06841542
 -0.0735998  -0.02130548  0.04605877 -0.08823569 -0.0147755   0.09354758
  0.02888806 -0.02228448  0.09114084  0.04604091  0.07873157 -0.05889153
  0.09564672  0.12373262 -0.00022965 -0.04361885  0.00578425 -0.03231039
  0.00238829  0.0007079   0.11109977 -0.00406435  0

NameError: name 'index_to_question_id_mapping' is not defined

In [1]:
import requests

In [19]:
query = 'adicionar transportadora nas notas fiscais geradas na rotina 1322'

In [119]:
query = 'Quais são os direitos dos trabalhadores?'

In [111]:
!python --version

Python 3.6.10


In [120]:
data = {
    'query': query,
    'k': '3',
    'threshold': '65'
}

response = requests.post(url='http://localhost:5000/query',
                                 json=data)
print(response)
print(response.text)
response.json()

<Response [200]>
{"response":[],"session_id":1}



{'response': [], 'session_id': 1}